In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder 

import warnings
warnings.filterwarnings('ignore')

In [20]:
df = pd.read_csv('subscription_df.csv')

### 대량이탈 현상 변수 추가 - Cohort Analysis (2019-06)

- Cohort Analysis 진행 시, 2019년 06월 2763건의 구독 거래가 이뤄진 뒤 1개월 후 99% 고객이 이탈하는 현상 발생
- 운영 기간 동안 구독 유입 인원이 30~50건 내외였기 때문에 해당 현상을 특이현상으로 규정할 필요가 있음

In [21]:
df_log = pd.read_csv('subscription_logs.csv')
df_user = pd.read_csv('users.csv')

df_log['subscription_start'] = pd.to_datetime(df_log['subscription_start'])
df_log['subscription_end'] = pd.to_datetime(df_log['subscription_start'])

sub_customers_6 = df_log[(df_log['subscription_start'].dt.year == 2019) & (df_log['subscription_start'].dt.month == 6)]['user_id'].unique() # 2019-06에 가입한 고객
sub_customers_7 = df_log[(df_log['subscription_start'].dt.year == 2019) & (df_log['subscription_start'].dt.month == 7)]['user_id'].unique() # 2019-07에 가입한 고객
churn_customers = list(set(sub_customers_6) - set(sub_customers_7))

df_user['chrun_19-06'] = df_user['user_id'].apply(lambda x: 1 if x in churn_customers else 0)
df_user = df_user[['user_id','chrun_19-06']]

In [22]:
df = pd.merge(df, df_user, on='user_id', how='inner')
df.head(3)

,user_id,account_id,profile_name,device_type,country,created_at,login_method,signup_duration,subscription_duration,payment_amount,...,content_view,recommended_view,category,watched_duration,bookmark_count,like_count,dislike_count,rating_count,total_interactions,chrun_19-06
0,2,4,Kevin,Mobile,China,2023-03-10 04:02:59,Password,19,18,190.0,...,105.0,30.0,Drama,109.400556,3.0,0.0,2.0,1.0,6.0,0
1,4,9,William,Web,Germany,2023-05-23 08:21:16,Social Login,17,1,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,6,4,Keith,Mobile,South Korea,2022-04-09 15:09:10,Social Login,30,29,300.0,...,124.0,23.0,Documentary,122.255000,2.0,3.0,3.0,1.0,9.0,0


In [23]:
df['chrun_19-06'].value_counts() # 1 이탈 / 0 미이탈 

chrun_19-06
1    2722
0    2721
Name: count, dtype: int64

In [24]:
df['churn'].value_counts()

churn
True     3272
False    2171
Name: count, dtype: int64

In [25]:
df.groupby('churn')['chrun_19-06'].value_counts() 

churn  chrun_19-06
False  0              2171
True   1              2722
       0               550
Name: count, dtype: int64

- 전체 이탈 비율 : 이탈 61%(3272명) 미이탈 39%(2171명)
- 전체 이탈 비율 (2019년 6월 이탈 사건 제외) : 이탈 20%(550명) 미이탈 80% (2171명)
<br/> &rarr; 전체 이탈의 83%(2722명)이 2019년 6월에 이탈


### 결측치 전처리

In [26]:
df.isna().sum()

user_id                     0
account_id                  0
profile_name                0
device_type                 0
country                     0
created_at                  0
login_method                0
signup_duration             0
subscription_duration       0
payment_amount              0
payment_method              0
churn                       0
content_view             2657
recommended_view         2657
category                 2657
watched_duration         2657
bookmark_count           2657
like_count               2657
dislike_count            2657
rating_count             2657
total_interactions       2657
chrun_19-06                 0
dtype: int64

- content_view, recommended_view, category, watched_duration 결측치는 고객이 구독했음에도 서비스를 이용하지 않은 것을 의미함
- bookmark_count, like_count, dislike_count, rating_count, total_interactions 결측치는 고객이 서비스에서 제공하는 북마크, 좋아요, 평가 기능을 이용하지 않을 것을 의미함
- content_view 와 total_interactions 결측치 수가 동일한 것으로 볼 때, 구독을 했지만 서비스를 이용하지 않은 고객이 2657명 이라는 것을 알 수 있음

In [27]:
df['category'].fillna('Unwatched', inplace=True)
df.fillna(0, inplace=True)

In [28]:
df.isna().sum().sum()

0

In [29]:
df.groupby('chrun_19-06')['category'].value_counts()

chrun_19-06  category   
0            Documentary    1214
             Movie           530
             Animation       510
             Drama           467
1            Unwatched      2657
             Documentary      25
             Drama            15
             Movie            15
             Animation        10
Name: count, dtype: int64

- 19년도 6월에 유입되고 이탈한 고객(2722명) 중 98%(2657명)는 컨텐츠 시청을 안 함

### 불필요한 피쳐 정리

- user_id, account_id, profile_name 등 유저 관련 정보는 이탈여부에 영향을 미치지 않음 
- created_at 등 가입 일자는 이탈여부와 관련이 없음 
- payment_amount 는 가격 변동이 없어 가격에 따른 이탈여부를 확인할 수 없어 subscription_duration 과 동일
- total_interactions 는 bookmark, like, dislike, rating 의 총합과 동일

In [30]:
df = df[['device_type','country','login_method','signup_duration','subscription_duration','payment_method',
        'content_view','recommended_view','category','watched_duration',        
        'bookmark_count','like_count','dislike_count','rating_count',                    
        'chrun_19-06','churn']]           

### 인코딩 (One-Hot Encoding)
- 범주형(categorical) 변수들은 명목형(nominal)이므로 One-Hot Encoding 기법 사용 

In [31]:
df = pd.get_dummies(df, columns=['device_type','country','login_method','payment_method','category'], drop_first=False)

In [32]:
df = df.astype('float')
df

,signup_duration,subscription_duration,content_view,recommended_view,watched_duration,bookmark_count,like_count,dislike_count,rating_count,chrun_19-06,...,login_method_Password,login_method_Social Login,payment_method_Credit Card,payment_method_Gift Card,payment_method_PayPal,category_Animation,category_Documentary,category_Drama,category_Movie,category_Unwatched
0,19.0,18.0,105.0,30.0,109.400556,3.0,0.0,2.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,17.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,30.0,29.0,124.0,23.0,122.255000,2.0,3.0,3.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,40.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,36.0,36.0,74.0,16.0,65.516111,1.0,2.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5438,50.0,50.0,108.0,21.0,109.924167,2.0,4.0,5.0,3.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5439,57.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5440,23.0,22.0,125.0,19.0,130.656944,9.0,3.0,5.0,4.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5441,13.0,13.0,185.0,38.0,172.525000,2.0,6.0,4.0,5.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [33]:
df.rename(columns={'country_South Korea':'country_South_Korea',
                   'country_United States':'country_United_States',
                   'login_method_Social Login':'login_method_Social_Login',
                   'payment_method_Credit Card':'payment_method_Credit_Card',
                   'payment_method_Gift Card':'payment_method_Gift_Card'},
                   inplace=True)

In [34]:
df.to_csv('final_df.csv', index=False)